In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm

In [13]:
Currencies = pd.read_csv("Dataset-TD5-csv.csv")

In [14]:
# Split dataset and calculate averages
GBPEUR = Currencies.iloc[:, :3]
SEKEUR = Currencies.iloc[:, 3:6]
CADEUR = Currencies.iloc[:, 6:]

GBPEUR['GBPEUR_AVG'] = GBPEUR.iloc[:, [1, 2]].mean(axis=1)
SEKEUR['SEKEUR_AVG'] = SEKEUR.iloc[:, [1, 2]].mean(axis=1)
CADEUR['CADEUR_AVG'] = CADEUR.iloc[:, [1, 2]].mean(axis=1)

In [15]:
# Work with the last 513 data points
GBPEUR = GBPEUR.iloc[-513:]
SEKEUR = SEKEUR.iloc[-513:]
CADEUR = CADEUR.iloc[-513:]

In [16]:
GBPEUR.head()

,GBPEUR Date,GBPEUR HIGH,GBPEUR LOW,GBPEUR_AVG
12416,8/31/2016 12:00,1.1783,1.1778,1.17805
12417,8/31/2016 12:15,1.1785,1.1778,1.17815
12418,8/31/2016 12:30,1.1796,1.1782,1.17890
12419,8/31/2016 12:45,1.1798,1.1789,1.17935
12420,8/31/2016 13:00,1.1796,1.1789,1.17925


In [17]:
SEKEUR.head()

,SEKEUR Date,SEKEUR HIGH,SEKEUR LOW,SEKEUR_AVG
12416,8/31/2016 12:00,0.10510,0.10506,0.105080
12417,8/31/2016 12:15,0.10510,0.10508,0.105090
12418,8/31/2016 12:30,0.10510,0.10505,0.105075
12419,8/31/2016 12:45,0.10512,0.10506,0.105090
12420,8/31/2016 13:00,0.10514,0.10508,0.105110


In [18]:
# Calculate returns Step 1
GBPEUR_returns = GBPEUR['GBPEUR_AVG'].pct_change().dropna()
SEKEUR_returns = SEKEUR['SEKEUR_AVG'].pct_change().dropna()
CADEUR_returns = CADEUR['CADEUR_AVG'].pct_change().dropna()

In [19]:
# Haar Transform Function Step 2
def haar_transform(data):
    n = len(data)
    output = np.zeros(n)
    while n > 1:
        n = n // 2
        for i in range(n):
            output[i] = (data[2 * i] + data[2 * i + 1]) / 2
            output[n + i] = (data[2 * i] - data[2 * i + 1]) / 2
        data[:n] = output[:n]
    return output

In [20]:
def multiresolution_correlation(haar1, haar2, level):
    length = 2 ** level
    truncated_haar1 = haar1[:length]
    truncated_haar2 = haar2[:length]
    return np.corrcoef(truncated_haar1, truncated_haar2)[0, 1]

In [21]:
# Correlation Matrix at Level Step 3
def correlation_matrix_at_level(level, haar_transforms):
    matrix_size = len(haar_transforms)
    correlation_matrix = np.zeros((matrix_size, matrix_size))
    for i in range(matrix_size):
        for j in range(i, matrix_size):
            if i == j:
                correlation_matrix[i, j] = 1
            else:
                correlation = multiresolution_correlation(
                    haar_transforms[i], haar_transforms[j], level)
                correlation_matrix[i, j] = correlation_matrix[j, i] = correlation
    return correlation_matrix

In [38]:
# Hurst Exponent Calculation
def Hurst(serie):
    T = len(serie)
    M2 = np.mean((np.abs(serie[1:] - serie[:-1]))**2) # absolute moment scale 1/N
    M2_prime = np.mean(np.abs((serie[2:] - serie[:-2]))**2) # absolute moment scale 2/N
    H_est = 0.5 * np.log2(M2_prime / M2)
    return H_est

In [39]:
# Applying our Haar Transform
gbpeur_haar = haar_transform(GBPEUR_returns.values)
sekeur_haar = haar_transform(SEKEUR_returns.values)
cadeur_haar = haar_transform(CADEUR_returns.values)

In [40]:
# Number of levels
num_levels = 9

haar_transforms = [gbpeur_haar, sekeur_haar, cadeur_haar]
correlation_matrices = [correlation_matrix_at_level(level, haar_transforms) for level in range(num_levels)]

In [41]:
# Hurst Exponents
hurst_gbpeur = Hurst(GBPEUR['GBPEUR_AVG'].values)
hurst_sekeur = Hurst(SEKEUR['SEKEUR_AVG'].values)
hurst_cadeur = Hurst(CADEUR['CADEUR_AVG'].values)

In [42]:
print("Hurst Exponents:", hurst_gbpeur, hurst_sekeur, hurst_cadeur)

Hurst Exponents: 0.6980759205067774 0.6583244952455628 0.6974012762332173


In [35]:
# High-Frequency Volatilities
vol_gbpeur_hf = np.std(GBPEUR_returns)
vol_sekeur_hf = np.std(SEKEUR_returns)
vol_cadeur_hf = np.std(CADEUR_returns)

In [36]:
volatility_gbpeur = [vol_gbpeur_hf * (2**j)**hurst_gbpeur for j in range(num_levels)]
volatility_sekeur = [vol_sekeur_hf * (2**j)**hurst_sekeur for j in range(num_levels)]
volatility_cadeur = [vol_cadeur_hf * (2**j)**hurst_cadeur for j in range(num_levels)]

In [37]:
print("Volatilities (GBPEUR):", volatility_gbpeur)
print("Volatilities (SEKEUR):", volatility_sekeur)
print("Volatilities (CADEUR):", volatility_cadeur)

Volatilities (GBPEUR): [0.0003563861551747842, 0.0005781794020294867, 0.0009380033878342624, 0.0015217601189183875, 0.0024688118290034145, 0.004005251400174237, 0.006497878287092208, 0.010541765800775352, 0.017102324988011865]
Volatilities (SEKEUR): [0.0001826054371687138, 0.00028819678033825246, 0.00045484617262845316, 0.0007178603470584736, 0.0011329621065094917, 0.0017880958880737124, 0.002822059878769063, 0.004453912126568097, 0.00702938069472965]
Volatilities (CADEUR): [0.0003058278629555568, 0.0004959247158512311, 0.0008041822004552435, 0.0013040467451173353, 0.002114617697940157, 0.003429024323847673, 0.005560441409807847, 0.009016707305596634, 0.014621323136576174]
